# Illustration of the doubling-distance code
This notebook aims at illustrating how a doubling distance code like 

![image.png](./doubling_distance.png)

can be implemented using the template approach.

## 1. Define the building blocks

In [2]:
import numpy
from tqec.templates.scalable.rectangle import ScalableRectangle
from tqec.templates.fixed.base import FixedRaw
from tqec.templates.scalable.square import ScalableAlternatingSquare
from tqec.templates.fixed.square import FixedAlternatingSquare
from tqec.templates.base import TemplateWithIndices


left_white_scalable_rectangle = TemplateWithIndices(
    ScalableRectangle(1, 4), [1, 0]
)  # 2, 9, 19
right_white_scalable_rectangle = TemplateWithIndices(
    ScalableRectangle(1, 4), [0, 1]
)  # 4, 13, 23
top_white_scalable_rectangle = TemplateWithIndices(ScalableRectangle(4, 1), [1, 0])  # 8
top_black_scalable_rectangle = TemplateWithIndices(ScalableRectangle(4, 1), [0, 1])  # 1
bottom_black_scalable_rectangle = TemplateWithIndices(
    ScalableRectangle(4, 1), [1, 0]
)  # 24, 26
scalable_square = TemplateWithIndices(
    ScalableAlternatingSquare(4), [1, 1]
)  # 3, 10, 20, 22
fixed_square = TemplateWithIndices(FixedAlternatingSquare(2), [1, 1])  # 16
horizontal_scalable_rectangle = TemplateWithIndices(
    ScalableRectangle(4, 2), [1, 1]
)  # 6, 15, 17
vertical_scalable_rectangle = TemplateWithIndices(
    ScalableRectangle(2, 4), [1, 1]
)  # 11, 21
left_fixed_square = TemplateWithIndices(FixedRaw([[0]]), [1])
right_fixed_square = TemplateWithIndices(FixedRaw([[0]]), [1])
bottom_fixed_square = TemplateWithIndices(FixedRaw([[0]]), [1])
# Specific cases:
# - the corner numbered 7
corner = TemplateWithIndices(FixedRaw([[0]]), [7])


# - the square numbered 12 that has a specific top-right plaquette
class SpecificScalableAlternatingSquare(ScalableAlternatingSquare):
    def instanciate(
        self, x_plaquette: int, z_plaquette: int, special_plaquette: int, *_: int
    ) -> numpy.ndarray:
        arr = super().instanciate(x_plaquette, z_plaquette, *_)
        arr[0, -1] = special_plaquette
        return arr


top_right_specific_square = TemplateWithIndices(
    SpecificScalableAlternatingSquare(4), [1, 1, 1]
)

In [3]:
templates = [
    # 0
    top_black_scalable_rectangle,
    left_white_scalable_rectangle,
    scalable_square,
    right_white_scalable_rectangle,
    left_fixed_square,
    # 5
    horizontal_scalable_rectangle,
    corner,
    top_white_scalable_rectangle,
    left_white_scalable_rectangle,
    scalable_square,
    # 10
    vertical_scalable_rectangle,
    top_right_specific_square,
    right_white_scalable_rectangle,
    left_fixed_square,
    horizontal_scalable_rectangle,
    # 15
    fixed_square,
    horizontal_scalable_rectangle,
    right_fixed_square,
    left_white_scalable_rectangle,
    scalable_square,
    # 20
    vertical_scalable_rectangle,
    scalable_square,
    right_white_scalable_rectangle,
    bottom_black_scalable_rectangle,
    bottom_fixed_square,
    # 25
    bottom_black_scalable_rectangle,
]
assert len(templates) == 26

In [4]:
from tqec.enums import ABOVE_OF, BELOW_OF, LEFT_OF, RIGHT_OF, CornerPositionEnum

relations = [
    (0, ABOVE_OF, 2),
    (1, LEFT_OF, 2),
    (3, RIGHT_OF, 2),
    (4, BELOW_OF, 1),
    (5, BELOW_OF, 2),
    (9, BELOW_OF, 5),
    (8, LEFT_OF, 9),
    (10, RIGHT_OF, 9),
    (11, RIGHT_OF, 10),
    (12, RIGHT_OF, 11),
    (7, ABOVE_OF, 11),
    (13, BELOW_OF, 8),
    (14, BELOW_OF, 9),
    (15, BELOW_OF, 10),
    (16, BELOW_OF, 11),
    (19, BELOW_OF, 14),
    (18, LEFT_OF, 19),
    (20, RIGHT_OF, 19),
    (21, RIGHT_OF, 20),
    (22, RIGHT_OF, 21),
    (17, ABOVE_OF, 22),
    (23, BELOW_OF, 19),
    (24, RIGHT_OF, 23),
    (25, BELOW_OF, 21),
]
corner_relations = [
    ((6, CornerPositionEnum.LOWER_LEFT), (CornerPositionEnum.UPPER_RIGHT, 9)),
]

In [5]:
from tqec.templates.orchestrator import TemplateOrchestrator


doubling_template = TemplateOrchestrator(templates)
for start, reldir, end in relations:
    doubling_template.add_relation(start, reldir, end)
    
for (start, start_corner), (end_corner, end) in corner_relations:
    doubling_template.add_corner_relation((start, start_corner), (end, end_corner))

In [6]:
from tqec.display import display
display(doubling_template)

  .  .  1  .  1  .  .  .  .  .  .  .
  1  1  1  1  1  .  .  .  .  .  .  .
  .  1  1  1  1  1  .  .  .  .  .  .
  1  1  1  1  1  .  .  .  .  .  .  .
  .  1  1  1  1  1  .  .  .  .  .  .
  1  1  1  1  1  .  .  .  .  .  .  .
  .  1  1  1  1  7  .  1  .  1  .  .
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  .  1  .  1  .  1  .  1  .  1  .  .


In [7]:
display(doubling_template.scale_to(6))

  .  .  1  .  1  .  1  .  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  7  .  1  .  1  .  1  .  .
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  